In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./kit.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_type_code = df['Kit_Type_Code'].tolist()
list_part_key = df['Kit_Part_Key'].tolist()

print('总数量：' + str(len(list_part_key)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

from urllib.parse import quote
import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for type_code, part_key in zip(list_type_code, list_part_key):
    work.put_nowait((type_code, part_key))

def crawler():
    global df
    
    while not work.empty():
        type_code, part_key = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                if part_key == '':
                        df_temp = pd.DataFrame([{'status': 'ok',
                                                 'Part_Key': part_key,
                                                 'Vehicle': ''}])

                        status = 'ok'

                        break
                
                data = 'func=getbuyersguide&payload=' + quote('{"partData":{"listing_data_essential":{"parttype":"') + type_code + quote('","partkey":"') + part_key + quote('"}}}') + '&api_json_request=1&sctchecked=1&scbeenloaded=true&curCartGroupID='

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/x-www-form-urlencoded; charset=UTF-8'
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_vehicle = html.xpath('//div[contains(@class, "buyersguide-nested")]/descendant::table[contains(@class, "nobmp")]/tr')

                vehicle = '\n'.join([' '.join(_.xpath('./td/text()')) for _ in list_vehicle])

                # = = = = = = = = = = = = = = =
           
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Kit_JOIN': type_code + ';' + part_key,
                                         'Kit_Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(0.5)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Kit_JOIN': type_code + ';' + part_key}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(type_code + ', ' + part_key +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./kit_vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：869

5736, 8308132  <->  [ok] - 剩余数量：859
5723, 8314844  <->  [ok] - 剩余数量：858
11860, 8308296  <->  [ok] - 剩余数量：857
5724, 8306448  <->  [ok] - 剩余数量：856
5728, 8307424  <->  [ok] - 剩余数量：855
11859, 8921276  <->  [ok] - 剩余数量：854
5728, 8308292  <->  [ok] - 剩余数量：853
5728, 9659060  <->  [ok] - 剩余数量：852
5722, 8314860  <->  [ok] - 剩余数量：851
5728, 9659096  <->  [ok] - 剩余数量：850
5736, 9659092  <->  [ok] - 剩余数量：849
5723, 9659204  <->  [ok] - 剩余数量：848
5724, 9659040  <->  [ok] - 剩余数量：847
5728, 9659056  <->  [ok] - 剩余数量：846
5722, 9659216  <->  [ok] - 剩余数量：845
5722, 9659212  <->  [ok] - 剩余数量：844
5728, 9659100  <->  [ok] - 剩余数量：843
5722, 9659220  <->  [ok] - 剩余数量：842
5736, 9659088  <->  [ok] - 剩余数量：841
5724, 10110012  <->  [ok] - 剩余数量：840
5736, 10110028  <->  [ok] - 剩余数量：839
5724, 10110008  <->  [ok] - 剩余数量：838
5728, 10110016  <->  [ok] - 剩余数量：837
5736, 10110036  <->  [ok] - 剩余数量：836
5568, 8527968  <->  [ok] - 剩余数量：835
5728, 10110040  <->  [ok] - 剩余数量：834
5723, 10454636  <->  [ok] - 剩余数量：833
5589, 8921

5736, 11158205  <->  [ok] - 剩余数量：633
5723, 12804305  <->  [ok] - 剩余数量：632
5736, 14047813  <->  [ok] - 剩余数量：631
5728, 14047829  <->  [ok] - 剩余数量：630
5728, 13570117  <->  [ok] - 剩余数量：629
5724, 13570017  <->  [ok] - 剩余数量：628
5728, 14047833  <->  [ok] - 剩余数量：627
5728, 12804165  <->  [ok] - 剩余数量：626
5724, 12902169  <->  [ok] - 剩余数量：625
5736, 12804173  <->  [ok] - 剩余数量：624
5723, 13060845  <->  [ok] - 剩余数量：623
5723, 14047941  <->  [ok] - 剩余数量：622
5728, 11492509  <->  [ok] - 剩余数量：621
5728, 12804129  <->  [ok] - 剩余数量：620
5728, 13570105  <->  [ok] - 剩余数量：619
5728, 13570101  <->  [ok] - 剩余数量：618
5728, 11492533  <->  [ok] - 剩余数量：617
5723, 14047949  <->  [ok] - 剩余数量：616
5723, 11492625  <->  [ok] - 剩余数量：615
5722, 14047957  <->  [ok] - 剩余数量：614
5728, 11492521  <->  [ok] - 剩余数量：613
5736, 11492529  <->  [ok] - 剩余数量：612
5722, 8313840  <->  [ok] - 剩余数量：611
5724, 12020425  <->  [ok] - 剩余数量：610
5723, 8313692  <->  [ok] - 剩余数量：609
5728, 12020533  <->  [ok] - 剩余数量：608
13625, 11492505  <->  [ok] - 剩余数量：607
57

5724, 8304692  <->  [ok] - 剩余数量：405
5728, 7096488  <->  [ok] - 剩余数量：404
5724, 7093396  <->  [ok] - 剩余数量：403
5723, 8314756  <->  [ok] - 剩余数量：402
5722, 8314664  <->  [ok] - 剩余数量：401
5736, 8307912  <->  [ok] - 剩余数量：400
5722, 8314680  <->  [ok] - 剩余数量：399
5728, 8307108  <->  [ok] - 剩余数量：398
10465, 10195124  <->  [ok] - 剩余数量：397
5736, 8307944  <->  [ok] - 剩余数量：396
5728, 8307948  <->  [ok] - 剩余数量：395
5728, 10195092  <->  [ok] - 剩余数量：394
5722, 8314536  <->  [ok] - 剩余数量：393
5722, 8314780  <->  [ok] - 剩余数量：392
5736, 8307972  <->  [ok] - 剩余数量：391
5722, 10195128  <->  [ok] - 剩余数量：390
5724, 8305996  <->  [ok] - 剩余数量：389
5736, 8307940  <->  [ok] - 剩余数量：388
5723, 8314748  <->  [ok] - 剩余数量：387
5722, 8314772  <->  [ok] - 剩余数量：386
5722, 7088216  <->  [ok] - 剩余数量：385
5722, 10195120  <->  [ok] - 剩余数量：384
5728, 8307296  <->  [ok] - 剩余数量：383
5722, 8314744  <->  [ok] - 剩余数量：382
5728, 8307920  <->  [ok] - 剩余数量：381
5736, 8307968  <->  [ok] - 剩余数量：380
5723, 8314696  <->  [ok] - 剩余数量：379
5728, 8307300  <->  [ok

5722, 8314704  <->  [ok] - 剩余数量：178
5722, 7304776  <->  [ok] - 剩余数量：177
5722, 8314540  <->  [ok] - 剩余数量：176
5723, 8314228  <->  [ok] - 剩余数量：175
5723, 8314656  <->  [ok] - 剩余数量：174
5722, 8314488  <->  [ok] - 剩余数量：173
5722, 8314636  <->  [ok] - 剩余数量：172
5723, 8314716  <->  [ok] - 剩余数量：171
5736, 10110024  <->  [ok] - 剩余数量：170
5724, 8306424  <->  [ok] - 剩余数量：169
5722, 8314720  <->  [ok] - 剩余数量：168
5722, 7305852  <->  [ok] - 剩余数量：167
5722, 8314836  <->  [ok] - 剩余数量：166
5724, 8306428  <->  [ok] - 剩余数量：165
5723, 8314816  <->  [ok] - 剩余数量：164
5724, 8306420  <->  [ok] - 剩余数量：163
5722, 8313596  <->  [ok] - 剩余数量：162
5736, 8308280  <->  [ok] - 剩余数量：161
5723, 8314820  <->  [ok] - 剩余数量：160
5568, 8306432  <->  [ok] - 剩余数量：159
5728, 8308128  <->  [ok] - 剩余数量：158
5728, 8307120  <->  [ok] - 剩余数量：157
5568, 8306456  <->  [ok] - 剩余数量：156
5589, 8529060  <->  [ok] - 剩余数量：155
5722, 8314652  <->  [ok] - 剩余数量：154
5723, 8314904  <->  [ok] - 剩余数量：153
5728, 9659076  <->  [ok] - 剩余数量：152
11859, 8307316  <->  [ok] -